In [ ]:
!pip install argilla fastapi uvicorn

In [ ]:
import argilla as rg

client = rg.Argilla(
    api_url="https://robinhad-my-argilla.hf.space",
    api_key="",
)

In [9]:
dataset = client.datasets(name="pararook_eval_100_sent")

In [ ]:
from random import randint

debug = False
for record in dataset.records(
    with_suggestions=True,
    with_responses=True,
):
    if debug:
        print(record)
        # Access the record properties
        print(record.fields["eng_Latn"])
        print(record.metadata)
        print(record.vectors)
        print(record.suggestions)
        print(record.responses)

    print("Suggestions:")
    print(record.suggestions)

    if len(record.suggestions) == 0:
        record.suggestions.add(
            rg.Suggestion("rating_0", randint(1,10), score=0.9, agent="random")
        )

    dataset.records.log(records=[
        record # here it would be more efficient to log in batches, for illustration purposes we log one by one
    ])

In [ ]:
# my_webhook.py file
import argilla as rg

from datetime import datetime
from argilla import webhook_listener

@webhook_listener(events="record.updated")
async def my_webhook_handler(dataset: rg.Dataset, type: str, timestamp: datetime):
    for record in dataset.records(
        with_suggestions=True,
        with_responses=False,
    ):
        if debug:
            print(record)
            # Access the record properties
            print(record.fields["eng_Latn"])
            print(record.metadata)
            print(record.vectors)
            print(record.suggestions)
            print(record.responses)

        print("Suggestions:")
        print(record.suggestions)

        if len(record.suggestions) == 0:
            record.suggestions.add(
                rg.Suggestion("rating_0", randint(1,10), score=0.9, agent="random")
            )

        dataset.records.log(records=[
            record # here it would be more efficient to log in batches, for illustration purposes we log one by one
        ])
    print(dataset, type, timestamp)

In [ ]:
# my_webhook.py file
from argilla import get_webhook_server

server = get_webhook_server()

In [ ]:
!uvicorn my_webhook:server

In [ ]:


records = [
    rg.Record(
        fields={
            "question": "Do you need oxygen to breathe?",
            "answer": "Yes"
        },
    ),
    rg.Record(
        fields={
            "question": "What is the boiling point of water?",
            "answer": "100 degrees Celsius"
        },
    ), # 
]

dataset.records.log(records)

In [ ]:
# Add records to the dataset with the label 'my_label'
records = [
    rg.Record(
        fields={
            "question": "Do you need oxygen to breathe?",
            "answer": "Yes"
        },
        suggestions=[
            rg.Suggestion(
                "my_label",
                "positive",
                score=0.9,
                agent="model_name"
            )
        ],
    ),
    rg.Record(
        fields={
            "question": "What is the boiling point of water?",
            "answer": "100 degrees Celsius"
        },
        suggestions=[
            rg.Suggestion(
                "my_label",
                "negative",
                score=0.9,
                agent="model_name"
            )
        ],
    ),
]
dataset.records.log(records)

In [ ]:
updated_records = []

for record in dataset.records():

    record.metadata["my_metadata"] = "new_value"
    record.metadata["my_new_metadata"] = "new_value"

    updated_records.append(record)

dataset.records.log(records=updated_records)